# 1. Prepare dirs and datasets

### 1. Upload dataset in dir /kaggle/input/imagenet/
### 2. Upload repo with code as dataset in dir /kaggle/input/project25/

In [26]:
!rm -R /kaggle/working/*
!cp -R /kaggle/input/project25/* /kaggle/working/
!mkdir -p /kaggle/working/datasets/ImageNet/val
!cp -R /kaggle/input/imagenet/val_small/* /kaggle/working/datasets/ImageNet/val

# 2. Install env

In [7]:
!pip install -q poetry

In [1]:
!poetry -q install

In [2]:
from kaggle_secrets import UserSecretsClient
import wandb
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api")
wandb.login(key=secret_value_0)

# 3. WarmUp

In [3]:
!poetry run python train.py -m torch --log-vandb  --print-freq 100 --epochs 5 --batch-size 16

# 4. Check equal torch and manual ResNet50

## Manual 
(default use 8 bit, 32 is very slow)

In [5]:
# !poetry run python train.py -m manual --log-vandb  --print-freq 100 --epochs 100 --batch-size 16
!poetry run python train.py -m quantized -qb 8 -qr "-1, 1" --log-vandb  --print-freq 100 --epochs 200 --batch-size 16

## Torch original

In [4]:
!poetry run python train.py -m torch --log-vandb  --print-freq 100 --epochs 5 --batch-size 16

# 5. Start train

## 5.1 Manual

In [11]:
# !poetry run python train.py -m manual -qb 4 --log-vandb  --print-freq 100 --epochs 100 --batch-size 16

In [12]:
#!poetry run python train.py -m manual  --log-vandb  --print-freq 100 --epochs 100 --batch-size 16
# !poetry run python train.py -m manual -qb 2 --log-vandb  --print-freq 100 --epochs 100 --batch-size 16

In [13]:
# !poetry run python train.py -m manual -qb 1 --log-vandb  --print-freq 100 --epochs 100 --batch-size 16

## 5.2 Torch

In [5]:
# !poetry run python train.py -m torch  --log-vandb  --print-freq 100 --epochs 100 --batch-size 16

# 6 Best Quantize

In [14]:
# !poetry run python train.py -m quantized -qb 16 -qr "-10, 10" --log-vandb  --print-freq 100 --epochs 100 --batch-size 16

### Generate some quantized models in cur dir

In [6]:
!poetry run python train.py -m quantizedBest -qa qSigmoid -qb 8 -qpr 50 -qr "-10, 10" --log-vandb  --print-freq 100 --epochs 100 --batch-size 16

### Train each generated models. Models has a wandb tag  - models score. Use it to select best model 

In [7]:
import os
for file in os.listdir("."):
    if file.endswith(".pth"):
        pth = os.path.join(".", file)
        !poetry run python train.py -m custom --checkpoints-custom "{pth}" --log-vandb  --print-freq 100 --epochs 200 --batch-size 16